In [1]:
#!pip install autogluon
#!pip install pandas

  Using cached scikit_learn-1.2.2-cp39-cp39-win_amd64.whl (8.4 MB)
  Using cached pandas-1.5.3-cp39-cp39-win_amd64.whl (10.9 MB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached Pillow-9.5.0-cp39-cp39-win_amd64.whl (2.5 MB)
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached lightgbm-3.3.5-py3-none-win_amd64.whl (1.0 MB)
  Obtaining dependency information for protobuf!=3.19.5,>=3.15.3 from https://files.pythonhosted.org/packages/99/7b/1085be78abfb2b5f6467ccb316d6bbde59633e8f889c5ba605ebfd0a6ef8/protobuf-4.23.4-cp39-cp39-win_amd64.whl.metadata
  Using cached protobuf-3.20.2-cp39-cp39-win_amd64.whl (904 kB)
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\holwe\\AppData\\Roaming\\Python\\Python39\\site-packages\\google\\~upb\\_message.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection._split import _BaseKFold
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [45]:
seed_value = 42  # Replace with your desired seed value
random.seed(seed_value)
np.random.seed(seed_value)

In [47]:
x_train_b = pd.read_csv('cleaned_data_quarterly/B/x_train_b.csv')
y_train_b = pd.read_csv('cleaned_data_quarterly/B/train_b.csv')
x_test_b = pd.read_csv('cleaned_data_quarterly/B/x_test_b.csv')

x_train_b

,date_forecast,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,...,forecast_day,forecast_hour,calc_year,calc_month,calc_day,calc_hour,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3
0,2019-01-01 00:00:00,5.6,5.6,5.5,5.4,1.239,1.239,1.240,1.241,1226.4,...,1,0,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
1,2019-01-01 01:00:00,5.4,5.4,5.4,5.5,1.241,1.240,1.239,1.239,1157.6,...,1,1,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
2,2019-01-01 02:00:00,5.5,5.4,5.4,5.3,1.238,1.238,1.239,1.239,1087.6,...,1,2,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
3,2019-01-01 03:00:00,5.3,5.3,5.4,5.4,1.240,1.240,1.240,1.239,1016.4,...,1,3,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
4,2019-01-01 04:00:00,5.5,5.6,5.7,5.9,1.239,1.238,1.237,1.236,1029.0,...,1,4,NaN,NaN,NaN,NaN,-0.0000,-0.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26937,2023-04-30 19:00:00,4.6,4.6,4.5,4.5,1.275,1.276,1.277,1.278,1636.1,...,30,19,2023.0,4.0,29.0,7.0,0.7652,0.0,-0.0,-0.0
26938,2023-04-30 20:00:00,4.5,4.5,4.5,4.5,1.278,1.279,1.279,1.279,1747.8,...,30,20,2023.0,4.0,29.0,7.0,-0.0000,-0.0,-0.0,-0.0
26939,2023-04-30 21:00:00,4.5,4.5,4.5,4.5,1.279,1.279,1.279,1.279,1797.7,...,30,21,2023.0,4.0,29.0,7.0,-0.0000,-0.0,-0.0,-0.0
26940,2023-04-30 22:00:00,4.5,4.5,4.5,4.5,1.279,1.280,1.280,1.280,1534.3,...,30,22,2023.0,4.0,29.0,7.0,-0.0000,-0.0,-0.0,-0.0


In [52]:
x_train_b_combined = x_train_b.merge(y_train_b, left_on='date_forecast', right_on='time', how='left')
x_train_b_combined['observed'] = x_train_b_combined['calc_year'].isna().astype(int)

train_data_b = x_train_b_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour','date_forecast'], axis = 1)
x_test_b['observed'] = x_test_b['calc_year'].isna().astype(int)
test_data_b = x_test_b.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','date_forecast'], axis = 1)

In [54]:
test_data_b

,location,absolute_humidity_2m:gm3_Q0,absolute_humidity_2m:gm3_Q1,absolute_humidity_2m:gm3_Q2,absolute_humidity_2m:gm3_Q3,air_density_2m:kgm3_Q0,air_density_2m:kgm3_Q1,air_density_2m:kgm3_Q2,air_density_2m:kgm3_Q3,ceiling_height_agl:m_Q0,...,wind_speed_w_1000hPa:ms_Q3,forecast_year,forecast_month,forecast_day,forecast_hour,direct_rad_x_sun_elevation_Q0,direct_rad_x_sun_elevation_Q1,direct_rad_x_sun_elevation_Q2,direct_rad_x_sun_elevation_Q3,observed
0,B,4.3,4.3,4.3,4.3,1.283,1.283,1.283,1.283,912.3,...,-0.0,2023,5,1,0,-0.0000,-0.000000,-0.0000,-0.00000,0
1,B,4.3,4.3,4.2,4.2,1.283,1.283,1.283,1.283,NaN,...,-0.0,2023,5,1,1,-0.0000,-0.000000,-0.0000,-0.00000,0
2,B,4.2,4.2,4.1,4.1,1.283,1.283,1.283,1.282,1482.8,...,-0.0,2023,5,1,2,-0.0000,-0.000000,-0.0000,-0.00000,0
3,B,4.1,4.0,4.0,4.0,1.282,1.282,1.282,1.283,2237.7,...,-0.0,2023,5,1,3,-0.0000,0.000000,5.4626,20.74660,0
4,B,3.9,3.9,3.9,3.9,1.283,1.282,1.282,1.281,2323.2,...,-0.0,2023,5,1,4,56.0780,125.151596,243.3750,419.51752,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,B,8.3,8.3,8.4,8.4,1.197,1.198,1.198,1.199,3692.2,...,0.0,2023,7,3,19,160.1557,131.985000,100.5885,58.52950,0
716,B,8.4,8.5,8.6,8.6,1.200,1.200,1.201,1.202,3553.3,...,0.0,2023,7,3,20,31.5776,15.477500,6.7200,2.01120,0
717,B,8.7,8.8,8.8,8.9,1.203,1.203,1.204,1.205,3014.1,...,0.0,2023,7,3,21,0.0000,-0.000000,-0.0000,-0.00000,0
718,B,9.0,9.0,9.0,9.0,1.206,1.206,1.206,1.206,2318.0,...,0.0,2023,7,3,22,-0.0000,-0.000000,-0.0000,-0.00000,0


# Trying with validation set equal to half of the estimated data


In [56]:
def split_dataset(train_data, observed_column='observed'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set contains the last half of the rows where observed = 0,
    and the training set contains the rest.

    :param train_data: The original training dataset as a pandas DataFrame.
    :param observed_column: The name of the column that indicates if the row is observed.
    :return: A tuple (training_set, validation_set)
    """

    # Filter rows where observed = 0
    observed_zero = train_data[train_data[observed_column] == 0]

    # Split the filtered dataset into two
    training_set_estimated, validation_set = train_test_split(observed_zero,test_size=0.5, random_state=42)
    

    # Combine the first half of observed_zero with the rest of the data where observed != 0
    training_set = pd.concat([train_data[train_data[observed_column] != 0], training_set_estimated])

    return training_set, validation_set


train_data_b, val_data_b = split_dataset(train_data_b, 'observed')


In [57]:

train_data_b.drop(columns= ['index'], inplace= True)

In [59]:
test_data_b['snow_density:kgm3_Q0'] = np.nan
test_data_b['snow_density:kgm3_Q1'] = np.nan
test_data_b['snow_density:kgm3_Q2'] = np.nan
test_data_b['snow_density:kgm3_Q3'] = np.nan

In [63]:
predictor_b= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_b,tuning_data = val_data_b, verbosity = 2,presets='best_quality',excluded_model_types=['CatBoost'], time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231106_131405\"
Presets specified: ['best_quality']
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231106_131405\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   685.80 GB / 1022.87 GB (67.0%)
Train Data Rows:    26942
Train Data Columns: 189
Label Column: pv_measurement
Preprocessing data ...
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\core\utils\uti

In [66]:
predictor_b.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-20.868773,144.975595,1337.147372,0.001002,0.610116,3,True,19
1,ExtraTreesMSE_BAG_L2,-21.092483,139.956576,1005.370960,2.658664,37.758479,2,True,15
2,WeightedEnsemble_L2,-21.258979,129.189583,456.313696,0.001002,0.555237,2,True,10
3,RandomForestMSE_BAG_L2,-21.291455,139.880765,1123.885143,2.582853,156.272661,2,True,13
4,LightGBMXT_BAG_L1,-21.294390,63.834321,183.918137,63.834321,183.918137,1,True,3
5,XGBoost_BAG_L2,-21.394599,137.638018,980.203196,0.340106,12.590715,2,True,17
6,NeuralNetFastAI_BAG_L2,-21.449613,138.046673,1029.828113,0.748761,62.215631,2,True,16
7,LightGBM_BAG_L2,-21.615589,137.463033,976.479527,0.165121,8.867046,2,True,12
8,CatBoost_BAG_L2,-21.733773,137.400004,1025.473763,0.102092,57.861282,2,True,14
9,LightGBMXT_BAG_L2,-21.872652,137.767147,983.014149,0.469235,15.401668,2,True,11


In [69]:
nan_counts = x_train_b_combined.isna().sum()
print(nan_counts)

date_forecast                    0
absolute_humidity_2m:gm3_Q0      0
absolute_humidity_2m:gm3_Q1      0
absolute_humidity_2m:gm3_Q2      0
absolute_humidity_2m:gm3_Q3      0
                                ..
direct_rad_x_sun_elevation_Q3    0
time                             0
index                            0
pv_measurement                   0
observed                         0
Length: 197, dtype: int64


In [73]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b#print the DataFrame

c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a f

,pv_measurement
0,0.021633
1,0.051190
2,0.020101
3,3.999418
4,51.389675
...,...
715,46.949928
716,17.141407
717,1.484121
718,-0.196183


In [74]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b#print the DataFrame

c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
c:\Users\holwe\miniconda3\lib\site-packages\autogluon\tabular\models\fastainn\tabular_nn_fastai.py:190: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a f

,pv_measurement
0,-0.182750
1,-0.163155
2,-0.155448
3,1.503704
4,12.314628
...,...
715,69.798340
716,44.278572
717,17.382914
718,2.631103


,pv_measurement
2155,69.798340
2156,44.278572
2157,17.382914
2158,2.631103
2159,0.395749


In [76]:
import os
output_dir = 'autogluon'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the dataframes into the folder
y_pred_b.to_csv(os.path.join(output_dir, 'y_pred_b.csv'))

,prediction
0,1.016324
1,0.851993
2,1.456194
3,61.065281
4,306.626099
...,...
2155,69.798340
2156,44.278572
2157,17.382914
2158,2.631103
